In [ ]:
import time
import requests
import pandas as pd
import numpy as np
from tqdm import tqdm
from statsmodels.tsa.stattools import coint

BASE_URL = "https://api.binance.com"
INTERVAL = "15m"
LIMIT = 1000

def fetch_klines(symbol: str, interval="15m", lookback_days=30, sleep=0.12) -> pd.Series:
    """
    Fetch 15m klines with pagination (up to lookback_days).
    Returns close prices indexed by open_time.
    """
    total_needed = lookback_days * 96  # 96 bars/day for 15m
    end_time = int(pd.Timestamp.utcnow().timestamp() * 1000)

    data = []
    while len(data) < total_needed:
        params = {"symbol": symbol, "interval": interval, "limit": LIMIT, "endTime": end_time}
        r = requests.get(f"{BASE_URL}/api/v3/klines", params=params, timeout=20)
        r.raise_for_status()
        klines = r.json()
        if not klines:
            break

        data = klines + data
        end_time = klines[0][0] - 1
        time.sleep(sleep)

    df = pd.DataFrame(data, columns=[
        "open_time","open","high","low","close","volume",
        "close_time","qav","trades","tbbav","tbqav","ignore"
    ])
    df["open_time"] = pd.to_datetime(df["open_time"], unit="ms")
    df.set_index("open_time", inplace=True)
    df["close"] = df["close"].astype(float)

    return df["close"].sort_index()

def rank_cointegration_vs_btc(selected_symbols, lookback_days=30, min_obs=500):
    """
    Rank ALL selected coins vs BTC using Engle–Granger cointegration.
    Returns every coin with p-value/test_stat/n_obs, sorted by p-value ascending.
    """
    btc = fetch_klines("BTCUSDT", interval=INTERVAL, lookback_days=lookback_days)
    rows = []

    for sym in tqdm(selected_symbols):
        sym = sym.upper()
        if sym == "BTC":
            continue

        pair = sym + "USDT"

        try:
            coin = fetch_klines(pair, interval=INTERVAL, lookback_days=lookback_days)
        except Exception as e:
            rows.append({
                "coin": sym,
                "pair": f"{sym}-BTC",
                "pvalue": np.nan,
                "test_stat": np.nan,
                "n_obs": 0,
                "status": f"fetch_error: {e}"
            })
            continue

        xy = pd.concat([btc, coin], axis=1, join="inner").dropna()
        n = int(xy.shape[0])

        if n < min_obs:
            rows.append({
                "coin": sym,
                "pair": f"{sym}-BTC",
                "pvalue": np.nan,
                "test_stat": np.nan,
                "n_obs": n,
                "status": f"too_few_obs(<{min_obs})"
            })
            continue

        btc_aligned = xy.iloc[:, 0].values
        coin_aligned = xy.iloc[:, 1].values

        # near-constant protection
        if np.std(btc_aligned) < 1e-12 or np.std(coin_aligned) < 1e-12:
            rows.append({
                "coin": sym,
                "pair": f"{sym}-BTC",
                "pvalue": np.nan,
                "test_stat": np.nan,
                "n_obs": n,
                "status": "near_constant_series"
            })
            continue

        try:
            score, pvalue, _ = coint(coin_aligned, btc_aligned)
            rows.append({
                "coin": sym,
                "pair": f"{sym}-BTC",
                "pvalue": float(pvalue),
                "test_stat": float(score),
                "n_obs": n,
                "status": "ok"
            })
        except Exception as e:
            rows.append({
                "coin": sym,
                "pair": f"{sym}-BTC",
                "pvalue": np.nan,
                "test_stat": np.nan,
                "n_obs": n,
                "status": f"coint_error: {e}"
            })

    out = pd.DataFrame(rows)
    out = out.sort_values(["pvalue", "n_obs"], ascending=[True, False], na_position="last")
    return out

# -------- Example usage --------
selected_symbols = ["BTC","ETH","BNB","SOL","XRP","ADA","AVAX","DOGE","DOT","LINK"]  # your top-20 list

ranked = rank_cointegration_vs_btc(selected_symbols, lookback_days=30, min_obs=500)
print(ranked)
